In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news-machine-hack/Test.csv
/kaggle/input/fake-news-machine-hack/Train.csv
/kaggle/input/fake-news-machine-hack/sample submission.csv


In [2]:
train = pd.read_csv("/kaggle/input/fake-news-machine-hack/Train.csv")
test = pd.read_csv("/kaggle/input/fake-news-machine-hack/Test.csv")
samp = pd.read_csv("/kaggle/input/fake-news-machine-hack/sample submission.csv")

In [3]:
train.head()

,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports ...,abortion
1,2,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,2,The economic turnaround started at the end of ...,"economy,jobs"


In [4]:
train['Text'] = train['Text'] + ' ' +  train['Text_Tag'].fillna("").apply(lambda x : ' '.join(x.split(',')))
test['Text'] = test['Text'] + ' ' +  test['Text_Tag'].fillna("").apply(lambda x : ' '.join(x.split(',')))

In [5]:
target = 'Labels'

In [6]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")
import re

In [7]:
import re

def clean(x):
  x = re.sub(r'@\w+'," ",x)
  x = re.sub(r'[^a-zA-Z]'," ",x)
  x = re.sub(r' [a-zA-Z]{1} '," ",x)
  return x

merge = pd.concat([train,test]).reset_index(drop=True)
merge["description_word_len"] = merge.apply(lambda x:len(re.findall(r"\w+",x['Text'])),axis=1)
m = merge.groupby('Text_Tag')['description_word_len'].agg(['mean','max',"min"])
merge = merge.merge(m,on='Text_Tag',how="left")
# merge["Product_Description"] = merge.apply(lambda x:clean(x['Product_Description']),axis=1)

merge

,Labels,Text,Text_Tag,description_word_len,mean,max,min
0,1.0,Says the Annies List political group supports ...,abortion,13,17.778846,46.0,3.0
1,2.0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",28,28.000000,28.0,28.0
2,3.0,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,21,24.240000,515.0,6.0
3,1.0,Health care reform legislation is likely to ma...,health-care,14,20.617577,56.0,6.0
4,2.0,The economic turnaround started at the end of ...,"economy,jobs",12,19.948529,41.0,7.0
...,...,...,...,...,...,...,...
11502,NaN,Says his budget provides the highest state fun...,education,14,18.779026,52.0,4.0
11503,NaN,Ive been here almost every day. civil-rights c...,"civil-rights,crime,criminal-justice",11,17.000000,22.0,11.0
11504,NaN,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",26,26.000000,26.0,26.0
11505,NaN,Says an EPA permit languished under Strickland...,"environment,government-efficiency",21,21.000000,21.0,21.0


In [8]:
!pip install transformers
!pip install -U sentence-transformers

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 62 kB 265 kB/s eta 0:00:011
     |████████████████████████████████| 884 kB 960 kB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 3.5 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-py3-none-any.whl size=101181 sha256=ce3694ad1bc76f2d4e71a1ddc13141d53a1d9f165b8fadcf43c7839556915f3c
  Stored in directory: /root/.cache/pip/wheels/b5/94/b4/953a1fd26652702c88112a188346df4cead56f0e3971a6d653
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0
ERROR: After Octo

In [9]:
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
import tokenizers
print('TF version',tf.__version__)

TF version 2.3.0


In [10]:
from sentence_transformers import SentenceTransformer
sentence_embedder = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
100%|██████████| 1.31G/1.31G [00:31<00:00, 41.6MB/s]


In [11]:
%%time
sentence_embeddings = sentence_embedder.encode(merge.Text.values.tolist(),batch_size=64,show_progress_bar=True)


CPU times: user 32.5 s, sys: 14.3 s, total: 46.8 s
Wall time: 50.9 s


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics  import log_loss
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

In [17]:
data = pd.DataFrame(sentence_embeddings)
for i in ["Text_Tag","Labels","description_word_len","mean","max","min"]:
  data[i] = merge[i].values
data.head()

,0,1,2,3,4,5,6,7,8,9,...,1020,1021,1022,1023,Text_Tag,Labels,description_word_len,mean,max,min
0,-0.819478,0.553745,0.281095,0.483959,0.425502,0.377212,1.252342,-0.328106,-1.140721,0.278601,...,-0.539367,0.769265,-0.707399,-0.341294,abortion,1.0,13,17.778846,46.0,3.0
1,0.010476,0.964721,1.305634,-0.034877,0.474277,-0.898328,-1.100798,-0.039020,0.823545,-0.465004,...,-0.526546,-0.527923,-1.859096,-0.061935,"energy,history,job-accomplishments",2.0,28,28.000000,28.0,28.0
2,-1.140447,-0.794626,-0.289983,-1.464744,0.150040,-0.164812,-0.585933,1.033618,-0.524356,0.459435,...,-0.965426,-1.898334,0.173119,-0.242052,foreign-policy,3.0,21,24.240000,515.0,6.0
3,0.584749,0.532029,0.670361,0.512556,0.771463,-0.094614,0.287700,-0.281629,-0.467286,-0.025476,...,-0.029822,1.412071,0.784867,-0.546644,health-care,1.0,14,20.617577,56.0,6.0
4,0.061722,0.049816,-0.310454,0.585992,1.065714,-0.015555,-0.189076,-0.759746,0.832176,-0.242550,...,-0.096825,0.597541,-0.825019,-0.632962,"economy,jobs",2.0,12,19.948529,41.0,7.0


In [18]:
train = data[~data.Labels.isna()]
test = data[data.Labels.isna()]
test.drop("Labels",axis=1,inplace=True)

In [19]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,1020,1021,1022,1023,Text_Tag,Labels,description_word_len,mean,max,min
0,-0.819478,0.553745,0.281095,0.483959,0.425502,0.377212,1.252342,-0.328106,-1.140721,0.278601,...,-0.539367,0.769265,-0.707399,-0.341294,abortion,1.0,13,17.778846,46.0,3.0
1,0.010476,0.964721,1.305634,-0.034877,0.474277,-0.898328,-1.100798,-0.039020,0.823545,-0.465004,...,-0.526546,-0.527923,-1.859096,-0.061935,"energy,history,job-accomplishments",2.0,28,28.000000,28.0,28.0
2,-1.140447,-0.794626,-0.289983,-1.464744,0.150040,-0.164812,-0.585933,1.033618,-0.524356,0.459435,...,-0.965426,-1.898334,0.173119,-0.242052,foreign-policy,3.0,21,24.240000,515.0,6.0
3,0.584749,0.532029,0.670361,0.512556,0.771463,-0.094614,0.287700,-0.281629,-0.467286,-0.025476,...,-0.029822,1.412071,0.784867,-0.546644,health-care,1.0,14,20.617577,56.0,6.0
4,0.061722,0.049816,-0.310454,0.585992,1.065714,-0.015555,-0.189076,-0.759746,0.832176,-0.242550,...,-0.096825,0.597541,-0.825019,-0.632962,"economy,jobs",2.0,12,19.948529,41.0,7.0


In [23]:
train['Text_Tag'] = train['Text_Tag'].fillna("").apply(lambda x : ' '.join(x.split(',')))
test['Text_Tag'] = test['Text_Tag'].fillna("").apply(lambda x : ' '.join(x.split(',')))

In [ ]:
X = train.drop(["Labels"],axis=1)
Y = train[['Labels']]

params = {
    "od_type":"Iter",
    "od_wait":150,
    "iterations":25000,
    'learning_rate':0.02,
    "eval_metric":"Accuracy",
    "task_type":"GPU",
    "boosting_type":"Plain"
}

best_score = np.inf
scores = []

folds_large = KFold(n_splits=5,shuffle=True,random_state=1250)

for train_idx , test_idx in folds_large.split(X,Y):
  train_set = (X.iloc[train_idx],Y.iloc[train_idx])
  test_set = (X.iloc[test_idx],Y.iloc[test_idx])

  model = CatBoostClassifier(**params)
  model.fit(*train_set,
            cat_features = ["Text_Tag"],
            eval_set=[test_set],early_stopping_rounds=500,verbose=200)

  score = log_loss(test_set[1].values,model.predict_proba(test_set[0]))
  print(score)
  scores.append(score)
  

  if score < best_score:
    best_score = score
    best_model = model

  print("---"*50)

print(f"Mean Score : {np.array(scores).mean()}")
print(f"Min Score : {np.array(scores).min()}")
print(f"Max Score : {np.array(scores).max()}")

plt.plot(scores)
plt.plot(np.arange(len(scores)),[np.array(scores).mean()]*len(scores),)
plt.show()

model_large = best_model

In [29]:
pd.set_option("display.max_colwidth",100)
pd.set_option("display.max_rows",150)
m = pd.DataFrame(list(zip(X.columns,model_large.feature_importances_))).sort_values(1,ascending=False)
m

,0,1
32,32,0.904796
747,747,0.877003
1025,description_word_len,0.785462
987,987,0.738166
601,601,0.634172
...,...,...
118,118,0.000000
428,428,0.000000
437,437,0.000000
611,611,0.000000


In [ ]:
submission = pd.DataFrame(model_large.predict_proba(test))
submission.columns = [f'Class_{i}' for i in submission.columns]
submission.to_csv("Sub_v0.4.csv",index=False)
submission